# Run BLAST

In [1]:
file1 = 'planarian_transcriptome.fasta'
type1 = 'nucl' #or 'prot' if file1 is a proteome
id1 = 'pl' #2-character ID (e.g. 'hu' for human)

file2 = 'schisto_transcriptome.fasta'

type2 = 'nucl'
id2 = 'sc' #2-character ID (e.g. 'mo' for mouse)
!bash map_genes.sh --tr1 {file1} --t1 {type1} --n1 {id1} --tr2 {file2} --t2 {type2} --n2 {id2}

Running tblastx in both directions
^C


# Run SAMap

In [1]:
from samap.mapping import SAMAP
from samap.analysis import get_mapping_scores, GenePairFinder
from samalg import SAM

SAMap accepts file paths to unprocessed, raw `.h5ad` files. Alternatively, if you already have a processed `SAM` object, you can load them in directly.


Prior to running SAMap, you should have run `map_genes.sh`, which expects 2-character identifiers describing each species. For example, planarians and schistosomes might get `pl` and `sc` identifiers, respectively. `map_genes.sh` generates a `maps/` directory with the transcriptome mapping BLAST results deposited. The input species IDs and path to the `maps/` directory should be input into SAMap.

In [2]:
id1 = 'pl'
id2 = 'sc'

In [4]:
# passing in file names (SAMap will process the data with SAM and save the resulting objects to two `.h5ad` files.)
fn1 = '/media/storage/Dropbox/scrna/notebooks/data/planarian.h5ad' #processed data will be automatically saved to `/path/to/file/file1_pr.h5ad`
fn2 = '/media/storage/Dropbox/scrna/notebooks/data/schisto.h5ad' #processed data will be automatically saved to `/path/to/file/file2_pr.h5ad`
# runs SAMAP (f_maps should be the path to the 'maps' directory generated in the BLAST step above)
sm = SAMAP(fn1,fn2,id1,id2,f_maps = '/media/storage/Dropbox/scrna/notebooks/maps/')
samap = sm.run()

"""
# passing in already-processed SAM objects
sam1=SAM()
sam2=SAM()
sam1.load_data('/path/to/file1_pr.h5ad')
sam2.load_data('/path/to/file2_pr.h5ad')

sm = SAMAP(sam1,sam2,id1,id2,f_maps = 'maps/')
samap = sm.run()
"""

Preparing data 1 for SAMap.
Preparing data 2 for SAMap.
11630 `pl` genes and 7427 `sc` gene symbols match between the datasets and the BLAST graph.
Stitching SAM 0 and SAM 1
Found 88729 gene pairs
Recomputing PC projections with gene pair subsets...
Running hsnwlib
Using leiden_clusters and leiden_clusters cluster labels.
Out-neighbor smart expansion 1
Out-neighbor smart expansion 2
Indegree coarsening
0/3 (0, 7657) True
1/3 (20000, 7657) True
2/3 (40000, 7657) True
Concatenating SAM objects...
ITERATION: 0 
Average alignment score (A.S.):  0.4494758679567957 
Max A.S. improvement: 0.5895449249249848 
Min A.S. improvement: 0.0
Calculating gene-gene correlations in the homology graph...
Stitching SAM 0 and SAM 1
Found 36515 gene pairs
Recomputing PC projections with gene pair subsets...
Running hsnwlib
Using leiden_clusters and leiden_clusters cluster labels.
Out-neighbor smart expansion 1
Out-neighbor smart expansion 2
Indegree coarsening
0/3 (0, 7657) True
1/3 (20000, 7657) True
2/3 (

"\n# passing in already-processed SAM objects\nsam1=SAM()\nsam2=SAM()\nsam1.load_data('/path/to/file1_pr.h5ad')\nsam2.load_data('/path/to/file2_pr.h5ad')\n\nsm = SAMAP(sam1,sam2,id1,id2,f_maps = 'maps/')\nsamap = sm.run()\n"

To calculate alignment scores between cell types, we can use `get_mapping_scores`. This function will use the combined SAM object produced by SAMap to calculate alignment scores between cell types in the provided cell type annotation columns of `sam.adata.obs`. If no cell type annotations exist, the leiden clusters generated by SAM can be used (`k1=k2='leiden_clusters'`).

The resulting tables show the highest-scoring alignment scores for each cell type in organism 1 (`D1`) and organism 2 (`D2`), respectively.

In [5]:
k1 = 'cluster' #cell types annotation key in `sm.sam1.adata.obs`
k2 = 'tissue' #cell types annotation key in `sm.sam2.adata.obs`
D1,D2,MappingTable = get_mapping_scores(sm,k1,k2)

In [6]:
D1

pl_Neoblast: 0                 pl_Cathepsin+ cells: 10                  \
             Cluster Alignment score                 Cluster Alignment score   
0        sc_Neoblast        0.598764            sc_Cathepsin         0.58303   
1   sc_Tegument_prog     0.000372316        sc_Tegument_prog      0.00652464   
2      sc_Neural_KK7     0.000255347             sc_Tegument      0.00312608   
3     sc_Parenchymal     0.000238063             sc_Neoblast      0.00255907   
4       sc_Cathepsin     0.000176026            sc_Intestine      0.00213583   
5          sc_Neural     0.000154783               sc_Muscle      0.00204314   
6          sc_Muscle     0.000148423               sc_Neural      0.00149881   
7       sc_Intestine     3.65333e-06                sc_Gland     0.000367092   
8     sc_Flame cells     2.41592e-06          sc_Flame cells     0.000176834   
9        sc_Tegument               0          sc_Parenchymal     9.91723e-05   
10          sc_Gland               0           sc_Neural_KK7     5.22123e-05   

       pl_Muscle: 14                   pl_Epidermal: 2                  \
             Cluster Alignment score           Cluster Alignment score   
0          sc_Muscle        0.551705  sc_Tegument_prog        0.529709   
1       sc_Intestine       0.0221045       sc_Tegument       0.0110845   
2      sc_Neural_KK7        0.007516    sc_Parenchymal      0.00218263   
3        sc_Neoblast      0.00177825         sc_Neural     0.000533285   
4          sc_Neural     0.000827803       sc_Neoblast     0.000321612   
5       sc_Cathepsin     0.000182522      sc_Cathepsin     6.92936e-05   
6   sc_Tegument_prog     0.000160643         sc_Muscle     6.42681e-05   
7        sc_Tegument               0          sc_Gland     5.64936e-05   
8     sc_Parenchymal               0      sc_Intestine     2.62418e-05   
9           sc_Gland               0     sc_Neural_KK7               0   
10    sc_Flame cells               0    sc_Flame cells               0   

    pl_Intestine: 30                 pl_Protonephridia: 29                  \
             Cluster Alignment score               Cluster Alignment score   
0       sc_Intestine        0.423742        sc_Flame cells        0.384437   
1       sc_Cathepsin       0.0359549          sc_Cathepsin        0.010683   
2           sc_Gland       0.0266434             sc_Neural      0.00523998   
3        sc_Tegument       0.0072613           sc_Tegument     0.000344565   
4          sc_Muscle      0.00262176      sc_Tegument_prog     0.000295734   
5   sc_Tegument_prog      0.00165449        sc_Parenchymal     0.000211041   
6          sc_Neural     3.29212e-05             sc_Muscle      0.00011042   
7     sc_Parenchymal               0           sc_Neoblast     3.25973e-05   
8      sc_Neural_KK7               0         sc_Neural_KK7               0   
9        sc_Neoblast               0          sc_Intestine               0   
10    sc_Flame cells               0              sc_Gland               0   

       pl_Muscle: 13                  pl_Epidermal: 11                  \
             Cluster Alignment score           Cluster Alignment score   
0          sc_Muscle        0.377412       sc_Tegument         0.36336   
1          sc_Neural     0.000674606  sc_Tegument_prog       0.0916215   
2      sc_Neural_KK7     0.000261405      sc_Cathepsin       0.0410767   
3       sc_Intestine     0.000218814    sc_Parenchymal      0.00633025   
4       sc_Cathepsin     2.26788e-05         sc_Neural      0.00209702   
5   sc_Tegument_prog      1.7636e-05         sc_Muscle      0.00124177   
6        sc_Neoblast     1.39472e-05      sc_Intestine     0.000308751   
7        sc_Tegument               0          sc_Gland     0.000249147   
8     sc_Parenchymal               0       sc_Neoblast     3.57892e-05   
9           sc_Gland               0     sc_Neural_KK7               0   
10    sc_Flame cells               0    sc_Flame cells               0   

        pl_Neural: 8    

In [7]:
D2

sc_Neoblast                             sc_Cathepsin  \
                    Cluster Alignment score                  Cluster   
0            pl_Neoblast: 0        0.598764  pl_Cathepsin+ cells: 10   
1            pl_Neoblast: 5        0.271645   pl_Cathepsin+ cells: 4   
2           pl_Neoblast: 22        0.153925  pl_Cathepsin+ cells: 28   
3    pl_Cathepsin+ cells: 4       0.0461106          pl_Intestine: 6   
4           pl_Epidermal: 3       0.0427878    pl_Parapharyngeal: 12   
5           pl_Intestine: 6         0.03546  pl_Cathepsin+ cells: 15   
6              pl_Neural: 1       0.0304383  pl_Cathepsin+ cells: 17   
7              pl_Muscle: 7       0.0219356         pl_Intestine: 19   
8     pl_Parapharyngeal: 12       0.0121177         pl_Epidermal: 11   
9          pl_Intestine: 19      0.00796742         pl_Intestine: 30   
10           pl_Pharynx: 25      0.00608962  pl_Cathepsin+ cells: 39   
11    pl_Protonephridia: 26      0.00570168            pl_Neural: 32   
12            pl_Neural: 21      0.00364086    pl_Protonephridia: 29   
13    pl_Parapharyngeal: 38      0.00336135  pl_Cathepsin+ cells: 42   
14  pl_Cathepsin+ cells: 10      0.00255907  pl_Cathepsin+ cells: 31   
15  pl_Cathepsin+ cells: 15      0.00249855         pl_Epidermal: 24   
16            pl_Muscle: 14      0.00177825    pl_Protonephridia: 26   
17            pl_Neural: 36      0.00124665         pl_Intestine: 43   
18  pl_Cathepsin+ cells: 28        0.001225           pl_Pharynx: 25   
19            pl_Neural: 18      0.00112168  pl_Cathepsin+ cells: 41   
20            pl_Neural: 20     0.000850726             pl_Neural: 1   
21         pl_Epidermal: 35     0.000651279          pl_Epidermal: 3   
22             pl_Neural: 8     0.000459836           pl_Neoblast: 5   
23            pl_Neural: 23       0.0003554          pl_Neoblast: 22   
24          pl_Epidermal: 2     0.000321612    pl_Protonephridia: 40   
25            pl_Muscle: 16     0.000230289             pl_Neural: 8   
26            pl_Neural: 34     9.06908e-05            pl_Neural: 23   
27             pl_Neural: 9     8.80583e-05            pl_Neural: 18   
28  pl_Cathepsin+ cells: 39     5.27963e-05            pl_Neural: 20   
29  pl_Cathepsin+ cells: 17      5.2646e-05             pl_Muscle: 7   
30    pl_Protonephridia: 40     3.80496e-05            pl_Neural: 34   
31         pl_Epidermal: 11     3.57892e-05            pl_Neural: 36   
32    pl_Protonephridia: 29     3.25973e-05             pl_Neural: 9   
33  pl_Cathepsin+ cells: 31     2.85698e-05    pl_Parapharyngeal: 38   
34            pl_Neural: 32     1.49269e-05            pl_Muscle: 14   
35            pl_Muscle: 13     1.39472e-05           pl_Neoblast: 0   
36            pl_Neural: 33     7.26552e-06            pl_Neural: 21   
37         pl_Intestine: 43     4.86255e-06            pl_Muscle: 16   
38         pl_Intestine: 30               0          pl_Epidermal: 2   
39           pl_Pharynx: 27               0         pl_Epidermal: 35   
40         pl_Epidermal: 24               0            pl_Muscle: 13   
41  pl_Cathepsin+ cells: 42               0           pl_Pharynx: 37   
42  pl_Cathepsin+ cells: 41               0            pl_Neural: 33   
43           pl_Pharynx: 37               0           pl_Pharynx: 27   

                                  sc_Muscle                  \
   Alignment score                  Cluster Alignment score   
0          0.58303            pl_Muscle: 14        0.551705   
1         0.284357            pl_Muscle: 13        0.377412   
2         0.223917             pl_Muscle: 7        0.242052   
3        0.0990904            pl_Muscle: 16        0.146065   
4        0.0854291    pl_Parapharyngeal: 12       0.0135867   
5         0.075856  pl_Cathepsin+ cells: 17      0.00632711   
6        0.0687788             pl_Neural: 8      0.00518293   
7        0.0449842             pl_Neural: 9      0.00489054   
8        0.0410767  pl_Cathepsin+ cells: 15      0.00389578   
9        0

In [8]:
MappingTable

,pl_Cathepsin+ cells: 10,pl_Cathepsin+ cells: 15,pl_Cathepsin+ cells: 17,pl_Cathepsin+ cells: 28,pl_Cathepsin+ cells: 31,pl_Cathepsin+ cells: 39,pl_Cathepsin+ cells: 4,pl_Cathepsin+ cells: 41,pl_Cathepsin+ cells: 42,pl_Epidermal: 11,pl_Epidermal: 2,pl_Epidermal: 24,pl_Epidermal: 3,pl_Epidermal: 35,pl_Intestine: 19,pl_Intestine: 30,pl_Intestine: 43,pl_Intestine: 6,pl_Muscle: 13,pl_Muscle: 14,pl_Muscle: 16,pl_Muscle: 7,pl_Neoblast: 0,pl_Neoblast: 22,pl_Neoblast: 5,pl_Neural: 1,pl_Neural: 18,pl_Neural: 20,pl_Neural: 21,pl_Neural: 23,pl_Neural: 32,pl_Neural: 33,pl_Neural: 34,pl_Neural: 36,pl_Neural: 8,pl_Neural: 9,pl_Parapharyngeal: 12,pl_Parapharyngeal: 38,pl_Pharynx: 25,pl_Pharynx: 27,pl_Pharynx: 37,pl_Protonephridia: 26,pl_Protonephridia: 29,pl_Protonephridia: 40
sc_Cathepsin,0.583030,0.075856,0.068779,0.223917,0.008655,0.025269,0.284357,0.003994,0.009656,0.041077,0.000069,0.008217,0.002730,0.000060,0.044984,0.035955,0.006121,0.099090,0.000023,0.000183,0.000145,0.000446,0.000176,0.002140,0.002698,0.002748,0.000717,0.000577,0.000170,0.001382,0.014348,0.000000,0.000259,0.000253,0.001782,0.000194,0.085429,0.000186,0.006068,0.000000,0.000008,0.006553,0.010683,0.002122
sc_Flame cells,0.000177,0.000019,0.000000,0.000024,0.000000,0.000000,0.000016,0.000000,0.000000,0.000000,0.000000,0.004583,0.000017,0.000000,0.000000,0.000000,0.000000,0.000057,0.000000,0.000000,0.000000,0.000000,0.000002,0.000000,0.000000,0.000000,0.000037,0.000821,0.000299,0.000602,0.000000,0.000000,0.000000,0.000000,0.000012,0.000000,0.000000,0.000000,0.000000,0.000042,0.000000,0.120630,0.384437,0.035803
sc_Gland,0.000367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000022,0.000000,0.000000,0.000249,0.000056,0.000000,0.000281,0.000297,0.001328,0.026643,0.000000,0.000108,0.000000,0.000000,0.000145,0.000101,0.000000,0.000000,0.000000,0.000033,0.000050,0.000102,0.000000,0.000166,0.000000,0.000000,0.000123,0.000000,0.000322,0.000000,0.004721,0.152423,0.000138,0.000000,0.000000,0.000031,0.000000,0.000211
sc_Intestine,0.002136,0.000119,0.000000,0.000337,0.000000,0.000000,0.000123,0.000000,0.000000,0.000309,0.000026,0.000080,0.000229,0.000744,0.000993,0.423742,0.002867,0.000178,0.000219,0.022104,0.000539,0.003163,0.000004,0.000000,0.000157,0.000123,0.000812,0.000000,0.000118,0.000297,0.000000,0.000000,0.000000,0.000053,0.000681,0.000028,0.000009,0.000000,0.002742,0.000000,0.000000,0.000616,0.000000,0.000954
sc_Muscle,0.002043,0.003896,0.006327,0.001097,0.000026,0.002669,0.002104,0.000053,0.000017,0.001242,0.000064,0.000482,0.001390,0.000000,0.001050,0.002622,0.000024,0.001799,0.377412,0.551705,0.146065,0.242052,0.000148,0.000041,0.000482,0.002443,0.000791,0.000406,0.000069,0.001129,0.000519,0.000356,0.002112,0.003109,0.005183,0.004891,0.013587,0.000534,0.000313,0.000439,0.000000,0.000860,0.000110,0.000557
sc_Neoblast,0.002559,0.002499,0.000053,0.001225,0.000029,0.000053,0.046111,0.000000,0.000000,0.000036,0.000322,0.000000,0.042788,0.000651,0.007967,0.000000,0.000005,0.035460,0.000014,0.001778,0.000230,0.021936,0.598764,0.153925,0.271645,0.030438,0.001122,0.000851,0.003641,0.000355,0.000015,0.000007,0.000091,0.001247,0.000460,0.000088,0.012118,0.003361,0.006090,0.000000,0.000000,0.005702,0.000033,0.000038
sc_Neural,0.001499,0.001575,0.003603,0.000478,0.000191,0.000222,0.002710,0.000000,0.000214,0.002097,0.000533,0.045130,0.009913,0.000042,0.000773,0.000033,0.000026,0.000804,0.000675,0.000828,0.000164,0.001156,0.000155,0.000162,0.002845,0.185195,0.154325,0.260733,0.115261,0.208507,0.000474,0.066575,0.031143,0.001299,0.343862,0.122546,0.084106,0.000066,0.004537,0.023403,0.000910,0.005903,0.005240,0.000099
sc_Neural_KK7,0.000052,0.000043,0.000000,0.000000,0.000000,0.000000,0.000013,0.000000,0.000000,0.000000,0.000000,0.000000,0.000020,0.000000,0.000000,0.000000,0.000000,0.000022,0.000261,0.007516,0.000441,0.000398,0.000255,0.000490,0.000011,0.000314,0.000067,0.000006,0.000025,0.000010,0.043245,0.000000,0.000312,0.000121,0.010368,0.000027,0.000069,0.000000,0.000010,0.000000

SAMap provides a class to find gene pairs enriched in different cell type pairs. The method entails finding gene pairs that contribute positively to the cross-species correlation between cell types and are differentially expressed in their respective mapped cell types.

In [9]:
gpf = GenePairFinder(sm,k1=k1,k2=k2)

Finding cluster-specific markers in pl:cluster and sc:tissue.


/home/alec/anaconda3/envs/SAMap/lib/python3.7/site-packages/anndata/compat/__init__.py:215: FutureWarning: During AnnData slicing, found matrix at .uns['nnm'] that happens to be dimensioned at n_obs×n_obs (50562×50562).

These matrices should now be stored in the .obsp attribute.
This slicing behavior will be removed in anndata 0.8.
  FutureWarning,
... storing 'cluster' as categorical
/home/alec/anaconda3/envs/SAMap/lib/python3.7/site-packages/scanpy/tools/_rank_genes_groups.py:404: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/home/alec/anaconda3/envs/SAMap/lib/python3.7/site-packages/anndata/compat/__init__.py:215: FutureWarning: During AnnData slicing, found matrix at .uns['nnm'] that happens to be dimensioned at n_obs×n_obs (7657×7657).

These matrices should now be stored in the .obsp attribute.
This slicing behavior will be removed in anndata 0.8.
  FutureWarning,
... storing 'tissue' as categorical
/home/alec/anaconda3/e

`gpf.find_genes` can now be used to find gene pairs enriched in a cell type mapping.

In [10]:
n1 = 'Neoblast: 0' #cell type ID from organism 1 (must be present in `sam1.adata.obs[k1]`)
n2 = 'Neoblast' #cell type ID from organism 2 (must be present in `sam2.adata.obs[k1]`)
Gp,G1,G2 = gpf.find_genes(n1,n2)
#Gp are the gene pairs, G1 are the genes from organism 1, G2 are the genes from organism 2

In [11]:
Gp,G1,G2

(array(['pl_dd_Smed_v4_659_0_1;sc_Smp_179320',
        'pl_dd_Smed_v4_2787_0_1;sc_Smp_086860',
        'pl_dd_Smed_v4_648_0_1;sc_Smp_027920',
        'pl_dd_Smed_v4_2614_0_1;sc_Smp_086860',
        'pl_dd_Smed_v4_15019_0_1;sc_Smp_086860',
        'pl_dd_Smed_v4_5764_0_1;sc_Smp_032500',
        'pl_dd_Smed_v4_5764_0_1;sc_Smp_143490',
        'pl_dd_Smed_v4_5764_0_1;sc_Smp_172530',
        'pl_dd_Smed_v4_5764_0_1;sc_Smp_094140',
        'pl_dd_Smed_v4_7837_0_1;sc_Smp_082490',
        'pl_dd_Smed_v4_1484_0_1;sc_Smp_086860',
        'pl_dd_Smed_v4_5764_0_1;sc_Smp_054840',
        'pl_dd_Smed_v4_10594_0_1;sc_Smp_162370',
        'pl_dd_Smed_v4_4273_0_1;sc_Smp_009600',
        'pl_dd_Smed_v4_3708_0_1;sc_Smp_245030',
        'pl_dd_Smed_v4_5764_0_1;sc_Smp_037590',
        'pl_dd_Smed_v4_11487_0_1;sc_Smp_082490',
        'pl_dd_Smed_v4_756_0_1;sc_Smp_179320',
        'pl_dd_Smed_v4_12365_0_1;sc_Smp_086860',
        'pl_dd_Smed_v4_4712_0_1;sc_Smp_032500',
        'pl_dd_Smed_v4_2787_0_1;sc_Smp_

To get a table of enriched gene pairs from all cell type mappings that have an alignment score above some threshold (`thr`), you can use:

In [ ]:
gene_pairs = gpf.find_all(thr=0.1)

Calculating gene pairs for the mapping: pl;Cathepsin+ cells: 10 to sc;Cathepsin
Calculating gene pairs for the mapping: pl;Cathepsin+ cells: 28 to sc;Cathepsin
Calculating gene pairs for the mapping: pl;Cathepsin+ cells: 4 to sc;Cathepsin
Calculating gene pairs for the mapping: pl;Epidermal: 11 to sc;Tegument
Calculating gene pairs for the mapping: pl;Epidermal: 2 to sc;Tegument_prog
Calculating gene pairs for the mapping: pl;Epidermal: 24 to sc;Tegument
Calculating gene pairs for the mapping: pl;Epidermal: 3 to sc;Tegument_prog
Calculating gene pairs for the mapping: pl;Epidermal: 35 to sc;Tegument
Calculating gene pairs for the mapping: pl;Intestine: 30 to sc;Intestine
Calculating gene pairs for the mapping: pl;Intestine: 6 to sc;Tegument_prog
Calculating gene pairs for the mapping: pl;Muscle: 13 to sc;Muscle
Calculating gene pairs for the mapping: pl;Muscle: 14 to sc;Muscle
Calculating gene pairs for the mapping: pl;Muscle: 16 to sc;Muscle


In [ ]:
gene_pairs

# Saving/Loading SAMap

In [ ]:
from samap.utils import save_samap, load_samap
# save to a .pkl file
save_samap(sm,'path/to/file') #including the file ending (.pkl) is optional

In [ ]:
# load
sm = load_samap('path/to/file') #including the file ending (.pkl) is optional

# Visualizing SAMap results


Launching an interactive GUI (requires SAMGUI, see the README instructions in the [SAM](https://github.com/atarashansky/self-assembling-manifold) github repo):

In [ ]:
sm.gui()

To create a sankey plot, use `samap.analysis.sankey_plot`. This requires `holoviews` to be installed (`pip install holoviews`). It should already be installed if you're using the Docker image.

In [ ]:
from samap.analysis import sankey_plot
k1 = 'cluster' #cell types annotation key in `sam1.adata.obs`
k2 = 'tissue' #cell types annotation key in `sam2.adata.obs`
sankey_plot(sm,k1,k2)